<a href="https://colab.research.google.com/github/VENKATAISAP/sap-community-developer-challenge-eda-hana/blob/main/SAP_EDA_Challenge_Week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!python -V

Python 3.9.16


In [ ]:
!pip install hana-ml

In [4]:
import hana_ml
print("The package version: ", hana_ml.__version__)

The package version:  2.16.23031601


In [5]:
import pandas as pd

In [6]:
from hana_ml import dataframe as hdf

In [7]:
from hana_ml.dataframe import ConnectionContext

In [8]:
cc = ConnectionContext('e8c8bf75-fd07-426a-9c7a-7e0d4c6d8562.hana.trial-us10.hanacloud.ondemand.com',443,'','')
print(cc.hana_version())


4.00.000.00.1678185420 (fa/CE2022.42)


In [9]:
print(cc.sql("SELECT NOW() FROM DUMMY").collect().CURRENT_TIMESTAMP[0])

2023-03-21 05:32:58.465000


Open the cursor

In [10]:
mycursor = cc.connection.cursor()

Execute an SQL statement to create a table SEASONS in the database.

In [13]:
mycursor.execute('DROP TABLE "SEASONS"');

In [14]:
mycursor.execute (''' CREATE TABLE "SEASONS" ( "ID" INT PRIMARY KEY,   "DESC" VARCHAR(12) ) ''');

In [15]:
mycursor.execute('TRUNCATE TABLE "SEASONS"');

records_inserted=mycursor.executemany(operation='INSERT INTO "SEASONS" ("ID", "DESC") VALUES (?, ?)',
                                     list_of_parameters=
                                      ((1, "WINTER"),
                                      (2, "SPRING"),
                                      (3, "SUMMER"),
                                      (4, "FALL"))
                                     )
print("Records inserted: ", sum(records_inserted))

Records inserted:  4


In [16]:
mycursor.execute('SELECT * FROM "SEASONS"')
#Get the first record
result=mycursor.fetchone()
print("the first record", result)

#Get the second record
#result=mycursor.fetchtwo()
#print("the second record", result)

#Get the rest of records
result=mycursor.fetchall()
print("the rest of records", result)

the first record (1, 'WINTER')
the rest of records [(2, 'SPRING'), (3, 'SUMMER'), (4, 'FALL')]


Check tables created so far in the current database schema

In [17]:
print(f"Current database schema: {cc.get_current_schema()}")

Current database schema: DBADMIN


In [18]:
mycursor.execute(f''' SELECT TABLE_NAME FROM M_TABLES WHERE SCHEMA_NAME='{cc.get_current_schema()}'
''')

True

In [19]:
result=mycursor.fetchall()
print(f"{len(result)} table(s) in the schema {cc.get_current_schema()}: {[x[0] for x in result]}")

2 table(s) in the schema DBADMIN: ['SEASONS', 'MONTHS']


Pandas DataFrames and HANA DataFrames InterOperability

In [ ]:
cc.create_table("MONTHS", table_structure={"ID":"INT", "DESC":"VARCHAR(10)"})

In [20]:
cc.get_tables()

,TABLE_NAME
0,SEASONS
1,MONTHS


Create Pandas DataFrame with data describing calendar months.

In [21]:
months_df=pd.DataFrame( {"ID": range(1,13), "DESC": ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC'] })

In [22]:
display(months_df)

,ID,DESC
0,1,JAN
1,2,FEB
2,3,MAR
3,4,APR
4,5,MAY
5,6,JUN
6,7,JUL
7,8,AUG
8,9,SEP
9,10,OCT


In [23]:
display(months_df.style.hide(axis='index'))
# the additional index column is removed/hidden

ID,DESC
1,JAN
2,FEB
3,MAR
4,APR
5,MAY
6,JUN
7,JUL
8,AUG
9,SEP
10,OCT


Create HANA DataFrame from Pandas DataFrame

In [24]:
months_hdf = hdf.create_dataframe_from_pandas(connection_context=cc, pandas_df=months_df, table_name='MONTHS', primary_key='ID', force=True )
                                     

100%|██████████| 1/1 [00:00<00:00, 14.46it/s]


In [25]:
type(months_hdf)

hana_ml.dataframe.DataFrame

In [30]:
type(months_df)

pandas.core.frame.DataFrame

In [26]:
print(months_hdf.select_statement)

SELECT * FROM "MONTHS"


In [27]:
mycursor.execute(months_hdf.select_statement)

#Get all records
result=mycursor.fetchall()

#Display returned records
print(f"All {len(result)} records: {result}")

All 12 records: [(1, 'JAN'), (2, 'FEB'), (3, 'MAR'), (4, 'APR'), (5, 'MAY'), (6, 'JUN'), (7, 'JUL'), (8, 'AUG'), (9, 'SEP'), (10, 'OCT'), (11, 'NOV'), (12, 'DEC')]


In [28]:
months_hdf.collect()

,ID,DESC
0,1,JAN
1,2,FEB
2,3,MAR
3,4,APR
4,5,MAY
5,6,JUN
6,7,JUL
7,8,AUG
8,9,SEP
9,10,OCT


In [29]:
type(months_df)

pandas.core.frame.DataFrame

Using hana-ml approach to load data from a CSV file

In [35]:
url="https://raw.githubusercontent.com/SAP-samples/hana-ml-samples/main/Python-API/pal/datasets/bike.csv"
bikes_df = pd.io.parsers.read_csv(url)

In [36]:
display(bikes_df)

,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,hum,windspeed,cnt,days_since_2011
0,WINTER,2011,JAN,NO HOLIDAY,SAT,NO WORKING DAY,MISTY,8.175849,80.5833,10.749882,985,0
1,WINTER,2011,JAN,NO HOLIDAY,SUN,NO WORKING DAY,MISTY,9.083466,69.6087,16.652113,801,1
2,WINTER,2011,JAN,NO HOLIDAY,MON,WORKING DAY,GOOD,1.229108,43.7273,16.636703,1349,2
3,WINTER,2011,JAN,NO HOLIDAY,TUE,WORKING DAY,GOOD,1.400000,59.0435,10.739832,1562,3
4,WINTER,2011,JAN,NO HOLIDAY,WED,WORKING DAY,GOOD,2.666979,43.6957,12.522300,1600,4
...,...,...,...,...,...,...,...,...,...,...,...,...
726,WINTER,2012,DEC,NO HOLIDAY,THU,WORKING DAY,MISTY,3.945849,65.2917,23.458911,2114,726
727,WINTER,2012,DEC,NO HOLIDAY,FRI,WORKING DAY,MISTY,3.906651,59.0000,10.416557,3095,727
728,WINTER,2012,DEC,NO HOLIDAY,SAT,NO WORKING DAY,MISTY,3.906651,75.2917,8.333661,1341,728
729,WINTER,2012,DEC,NO HOLIDAY,SUN,NO WORKING DAY,GOOD,4.024151,48.3333,23.500518,1796,729


In [40]:
bikes_hdf = hdf.create_dataframe_from_pandas(connection_context=cc, pandas_df=bikes_df, table_name='BIKES',
                                      primary_key='days_since_2011',
                                      force=True
                                     )

100%|██████████| 1/1 [00:00<00:00,  6.15it/s]


In [41]:
display(bikes_hdf)

In [42]:
type(bikes_hdf)

hana_ml.dataframe.DataFrame

In [44]:
print(bikes_hdf.select_statement)

SELECT * FROM "BIKES"


In [45]:
bikes_hdf.collect().head(n=10)

,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,hum,windspeed,cnt,days_since_2011
0,WINTER,2011,JAN,NO HOLIDAY,SAT,NO WORKING DAY,MISTY,8.175849,80.5833,10.749882,985,0
1,WINTER,2011,JAN,NO HOLIDAY,SUN,NO WORKING DAY,MISTY,9.083466,69.6087,16.652113,801,1
2,WINTER,2011,JAN,NO HOLIDAY,MON,WORKING DAY,GOOD,1.229108,43.7273,16.636703,1349,2
3,WINTER,2011,JAN,NO HOLIDAY,TUE,WORKING DAY,GOOD,1.400000,59.0435,10.739832,1562,3
4,WINTER,2011,JAN,NO HOLIDAY,WED,WORKING DAY,GOOD,2.666979,43.6957,12.522300,1600,4
5,WINTER,2011,JAN,NO HOLIDAY,THU,WORKING DAY,GOOD,1.604356,51.8261,6.000868,1606,5
6,WINTER,2011,JAN,NO HOLIDAY,FRI,WORKING DAY,MISTY,1.236534,49.8696,11.304642,1510,6
7,WINTER,2011,JAN,NO HOLIDAY,SAT,NO WORKING DAY,MISTY,-0.245000,53.5833,17.875868,959,7
8,WINTER,2011,JAN,NO HOLIDAY,SUN,NO WORKING DAY,GOOD,-1.498349,43.4167,24.250650,822,8
9,WINTER,2011,JAN,NO HOLIDAY,MON,WORKING DAY,GOOD,-0.910849,48.2917,14.958889,1321,9


In [46]:
bikes_hdf.shape

[731, 12]

In [50]:
bikes_hdf.collect()["season"].unique().tolist()

['WINTER', 'SPRING', 'SUMMER', 'FALL']